# Recent  Approach : AthletePose3D - 3D Human Pose Estimation
Based on: Yeung et al. (2025) - AthletePose3D Dataset Paper

Implementation of methods from the paper:
- 2D Pose Detection (HRNet/ViTPose approach)
- 3D Pose Lifting (MotionAGFormer/TCPFormer)
- Kinematic Validation (Joint angles & velocities)
- Metrics: MPJPE, P-MPJPE, PDJ

Data  set : AthletePose3D: A Benchmark Dataset for 3D Human Pose Estimation and Kinematic Validation in Athletic Movements
Dataset Overview (Table 1 & 2 from Paper)

| Characteristic | Details |
|---------------|---------|
| **Total Frames** | 1.3 million frames |
| **Postures** | 165,000 individual postures |
| **Sports** | 3 categories, 12 motion types |
| **Athletes** | 8 athletes (amateur to professional) |
| **Keypoints** | 55/86 keypoints (varies by sport) |
| **Cameras** | 4-12 high-speed synchronized cameras |
| **Resolution** | 1920×1080 |

 Three Sport Categories :
[alt text](datset.jpg)


 1. Running (Lab Environment)
 2. Track & Field (Lab Environment)
 3. Figure Skating (Ice Rink)

we choose some running data  videos.

 # 1. Install Required Libraries 
What this does: Installs libraries mentioned in the paper

In [ ]:
import subprocess
import sys

print("="*70)
print(" INSTALLING LIBRARIES (Paper Technologies Only)")
print("="*70)

# Paper mentions these frameworks
packages = [
    'opencv-python',      # Video processing
    'numpy',              # Numerical computations
    'matplotlib',         # Visualization
    'scipy',              # Butterworth filter (Section 3.3)
    'torch',              # Deep learning framework
    'torchvision',        # Vision models
    'mediapipe',          # For demo (lightweight alternative)
    'pillow',             # Image processing
]

print("\nInstalling packages...")
for pkg in packages:
    print(f"  Installing {pkg}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("\n✓ Installation complete!")
print("="*70)

# 2. Import Libraries and Setup
 What this does: Imports all required modules

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.spatial.transform import Rotation
from mpl_toolkits.mplot3d import Axes3D
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("  AthletePose3D Implementation")
print("="*70)
print("\nPaper Information:")
print("  Title: AthletePose3D")
print("  Authors: Yeung et al. (2025)")
print("  Focus: 3D pose estimation for athletic movements")
print("  Dataset: 1.3M frames, 165K postures, 12 sports")
print("\nKey Technologies (from paper):")
print("  • 2D Models: HRNet, ViTPose, MogaNet (best: PDJ 95.7)")
print("  • 3D Models: MotionAGFormer, TCPFormer (best: MPJPE 98.26mm)")
print("  • Metrics: MPJPE, P-MPJPE, PDJ")
print("  • Kinematic: 4th-order Butterworth filter @ 8Hz")
print("  • Input: 81 frames per sequence")
print("  • Resolution: 1920×1080 @ 60/120 FPS")
print("="*70)

# 3. Load  Running Videos
What this does: Loads your 10-12 running videos

In [ ]:
VIDEO_FOLDER = "New_Paper"

print("\n" + "="*70)
print(" LOADING YOUR RUNNING VIDEOS")
print("="*70)

# Check if folder exists
if not os.path.exists(VIDEO_FOLDER):
    print(f"\n❌ ERROR: Folder '{VIDEO_FOLDER}' not found!")
    print("Please create the folder and add your running videos.")
    print("\nExpected structure:")
    print("  new_paper/")
    print("    ├── run1.mp4")
    print("    ├── run2.mp4")
    print("    └── ...")
else:
    # Find all video files
    video_files = []
    for ext in ['*.mp4', '*.avi', '*.mov', '*.MP4', '*.AVI', '*.MOV']:
        video_files.extend(Path(VIDEO_FOLDER).glob(ext))
    
    video_files = [str(f) for f in video_files]
    
    print(f"\n✓ Found {len(video_files)} videos:")
    for i, vf in enumerate(video_files, 1):
        # Get video info
        cap = cv2.VideoCapture(vf)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0
        cap.release()
        
        print(f"  {i}. {os.path.basename(vf)}")
        print(f"     Resolution: {width}x{height}, FPS: {fps:.1f}, "
              f"Frames: {frame_count}, Duration: {duration:.1f}s")
    
    print(f"\nPaper specifications:")
    print(f"  • Running: 120 FPS (your videos may vary)")
    print(f"  • Resolution: 1920×1080 (paper standard)")
    print(f"  • Athletes: Inter-university level")

print("="*70)


# 4.Load and Process Single Video
 What this does: Loads one video for processing

In [ ]:
if len(video_files) > 0:
    # Select first video
    selected_video = video_files[0]
    print(f"\n📹 Processing: {os.path.basename(selected_video)}")
    
    # Load video frames
    cap = cv2.VideoCapture(selected_video)
    frames = []
    
    print("Loading frames...")
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"  Loaded {frame_count} frames...", end='\r')
    
    cap.release()
    frames = np.array(frames)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"\n✓ Loaded {len(frames)} frames")
    print(f"  Shape: {frames.shape}")
    print(f"  FPS: {fps:.1f}")
    print(f"  Duration: {len(frames)/fps:.2f}s")
    
    # Display sample frames
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    sample_indices = [0, len(frames)//2, len(frames)-1]
    
    for ax, idx in zip(axes, sample_indices):
        ax.imshow(frames[idx])
        ax.set_title(f"Frame {idx}")
        ax.axis('off')
    
    plt.suptitle("Sample Frames from Your Running Video", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠️ No videos found. Please add videos to new_paper/ folder")


# 5. nitialize 2D Pose Detector (Paper Models)
 What this does: Sets up 2D pose estimation
 Paper Section 3.2: HRNet, ViTPose, MogaNet